In [ ]:
import autokeras

In [1]:
from keras.datasets import mnist
from autokeras.image.image_supervised import ImageClassifier

if __name__ == '__main__':
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))
    
    clf = ImageClassifier(path="~/automodels/",verbose=True, augment=False)
    clf.fit(x_train, y_train, time_limit=13 * 60)
    clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)
    y = clf.evaluate(x_test, y_test)
    print(y)
    

Using TensorFlow backend.


Saving Directory: ~/automodels/
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   0.064408078789711    |         0.9936         |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
Epoch-20, Current Metric - 0.988:  95%

In [8]:
import graphviz
clf.export_keras_model('autokeras_model1.h5')

In [9]:
from keras.models import load_model
model = load_model('autokeras_model1.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 1)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
activation_6 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 14, 14, 64)        256       
__________

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
import os
from graphviz import Digraph
import sys
from autokeras.utils import pickle_from_file

sys.path.append('/media/controlslab/DATA/Projects/automatic_model_selection/code/comparison/~/automodels')

def to_pdf(graph, path):
    dot = Digraph(comment='The Round Table')

    for index, node in enumerate(graph.node_list):
        dot.node(str(index), str(node.shape))

    for u in range(graph.n_nodes):
        for v, layer_id in graph.adj_list[u]:
            dot.edge(str(u), str(v), str(graph.layer_list[layer_id]))

    dot.render(path)

def visualize(path):
    cnn_module = pickle_from_file(os.path.join(path, 'module'))
    cnn_module.searcher.path = path
    for item in cnn_module.searcher.history:
        model_id = item['model_id']
        graph = cnn_module.searcher.load_model_by_id(model_id)
        to_pdf(graph, os.path.join(path, str(model_id)))


if __name__ == '__main__':
    visualize('~/automodels/')
    

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', '~/automodels/0'], make sure the Graphviz executables are on your systems' PATH